In [91]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz


In [92]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [93]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [94]:
fname_dir="/content/blood/"
fname_url="/content/drive/MyDrive/blood/Update_train.csv"
fname=fname_url.split("/")[-1]

In [95]:
fname_dir="/content/blood/"
!mkdir $fname_dir

mkdir: cannot create directory ‘/content/blood/’: File exists


In [96]:
!cp $fname_url $fname_dir

In [97]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [98]:
file_name=fname_dir+fname

In [99]:
import pandas as pd

In [100]:

df= pd.read_csv(file_name)
df.head()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,donation_id
0,0.520883,0.528200,0.529795,0.537382,0.540265,0.546260,0.550882,0.558865,0.571352,0.585775,0.600688,0.612949,0.617855,0.621710,0.622049,0.623373,0.620610,0.619479,0.616463,0.612414,0.609885,0.605408,0.602115,0.597710,0.593485,0.592332,0.588680,0.587396,0.585577,0.585244,0.583597,0.584460,0.584548,0.586147,0.589067,0.591360,0.595636,0.599054,0.602856,0.607817,...,1.468505,1.454910,1.434286,1.443249,1.417403,1.418581,1.403326,1.395943,1.386288,1.381532,1.372256,1.367083,1.362999,1.353013,1.345374,1.341283,1.323804,1.331071,1.332295,1.328928,1.309800,1.328064,1.329464,1.314664,1.313583,1.313329,1.312243,1.310330,1.353629,1.336609,1.319344,1.278294,0,0.0026,39.27,44.17,low,ok,high,2250
1,0.529109,0.534852,0.541075,0.544269,0.546594,0.555183,0.561367,0.567637,0.582427,0.595801,0.608619,0.619217,0.625162,0.627111,0.629326,0.628116,0.627815,0.623644,0.620762,0.617781,0.613032,0.610404,0.605528,0.602216,0.598417,0.595489,0.593692,0.591697,0.590110,0.589316,0.589131,0.587455,0.589060,0.589305,0.591252,0.594853,0.598545,0.601338,0.605673,0.610725,...,1.477741,1.457003,1.445143,1.439036,1.434363,1.422782,1.408831,1.405730,1.394905,1.386579,1.373887,1.376208,1.360633,1.358802,1.353009,1.344997,1.332883,1.324965,1.339512,1.336343,1.322957,1.311612,1.316437,1.316180,1.316063,1.320698,1.321068,1.286939,1.301804,1.314457,1.282723,1.267465,1,0.0026,39.39,44.09,low,ok,high,2250
2,0.528434,0.532036,0.540413,0.541964,0.549347,0.555902,0.561458,0.567824,0.579568,0.593048,0.607972,0.617558,0.624398,0.626222,0.627771,0.626445,0.623953,0.622616,0.618701,0.615181,0.611624,0.608587,0.605730,0.601288,0.597154,0.595654,0.592535,0.591893,0.589885,0.588528,0.587108,0.586997,0.589043,0.588888,0.591670,0.594962,0.597498,0.600767,0.604963,0.612166,...,1.468838,1.456374,1.450920,1.438182,1.434188,1.427201,1.413866,1.403336,1.404049,1.383698,1.378195,1.374761,1.373550,1.362671,1.350157,1.344038,1.341004,1.335655,1.333660,1.325491,1.335868,1.334114,1.327512,1.339627,1.322394,1.328299,1.311948,1.310330,1.311108,1.283048,1.287730,1.266464,2,0.0026,39.50,44.00,low,ok,high,2250
3,0.530528,0.531880,0.539558,0.542433,0.549097,0.556224,0.561398,0.566963,0.577525,0.591982,0.607109,0.616252,0.622437,0.625715,0.626799,0.625676,0.625084,0.622652,0.618510,0.614896,0.611060,0.608285,0.603976,0.600319,0.596856,0.594169,0.591334,0.589620,0.589549,0.587194,0.586685,0.587205,0.587368,0.588510,0.592029,0.593472,0.597904,0.601190,0.604864,0.610652,...,1.474065,1.456521,1.456408,1.445327,1.429233,1.422914,1.422161,1.405341,1.403060,1.390475,1.381461,1.370967,1.370843,1.358100,1.362090,1.352020,1.340746,1.345123,1.332574,1.345660,1.349603,1.329325,1.331542,1.347718,1.321111,1.320285,1.303363,1.306762,1.297632,1.314811,1.314408,1.342224,3,0.0026,39.60,43.89,low,ok,high,2250
4,0.527530,0.536424,0.538207,0.541504,0.550546,0.555203,0.560394,0.567352,0.579438,0.592049,0.606031,0.617109,0.624688,0.626810,0.628163,0.627580,0.625115,0.623415

In [101]:
df.describe()
df=df[df.donation_id!=6824]
df=df[df.donation_id!=7148]
df=df[df.donation_id!=6362]
df=df[df.donation_id!=4122]



In [102]:
df.describe()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,donation_id
count,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,...,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000,28920.000000
mean,0.508651,0.510847,0.514623,0.520249,0.526609,0.533964,0.540240,0.545640,0.553637,0.565879,0.578943,0.590577,0.597466,0.600780,0.602856,0.603231,0.601722,0.598961,0.595728,0.592548,0.589275,0.586249,0.583018,0.579489,0.576450,0.573381,0.570814,0.568910,0.567210,0.565860,0.565128,0.564873,0.565300,0.566338,0.567969,0.570353,0.573232,0.577075,0.580948,0.585060,...,1.484168,1.471509,1.460288,1.449381,1.437852,1.427042,1.415861,1.404398,1.394337,1.386746,1.374355,1.365617,1.357340,1.348685,1.341070,1.333946,1.326705,1.319977,1.313352,1.307159,1.302401,1.298144,1.294718,1.292297,1.291090,1.290035,1.288572,1.285285,1.279460,1.270504,1.262310,1.256160,1.253497,1.267463,1.253867,18453.483402,0.007550,41.211432,35.333558,6652.136929
std,0.037351,0.037289,0.037443,0.037749,0.037597,0.037438,0.037714,0.038005,0.038109,0.038412,0.038610,0.038752,0.038950,0.039030,0.039057,0.039004,0.038795,0.038565,0.038412,0.038358,0.038205,0.038167,0.038099,0.038078,0.038066,0.037958,0.037941,0.037982,0.037965,0.037930,0.037911,0.037928,0.037974,0.038030,0.038055,0.038157,0.038254,0.038444,0.038567,0.038608,...,0.078710,0.077475,0.076225,0.075055,0.073846,0.072804,0.071627,0.070856,0.070088,0.069666,0.068370,0.067744,0.067391,0.066721,0.066342,0.065891,0.065369,0.064935,0.064736,0.064279,0.064129,0.063969,0.063858,0.063966,0.064350,0.064863,0.066053,0.068376,0.072219,0.078854,0.086046,0.092041,0.096361,0.102609,0.102225,11491.523001,0.004636,2.754541,8.654283,3046.276623
min,0.398490,0.399354,0.406813,0.411516,0.419261,0.431098,0.430637,0.439161,0.447338,0.459307,0.473954,0.483997,0.492321,0.496650,0.498904,0.498780,0.497894,0.495571,0.493448,0.489953,0.487952,0.485595,0.482107,0.479002,0.476591,0.473782,0.472347,0.470869,0.468818,0.467783,0.466990,0.467501,0.467450,0.468256,0.469823,0.472030,0.474908,0.477831,0.481281,0.484260,...,1.221793,1.206966,1.196696,1.187510,1.178352,1.167833,1.160042,1

In [103]:
df["data"]=df["hdl_cholesterol_human"]+"+"+  df["hemoglobin(hgb)_human"]+"+"+df["cholesterol_ldl_human"]

In [104]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

_3_Label_encoder=LabelEncoder()
_3_Label_encoder.fit(df["data"])
_3_Label_encoder.classes_
y=_3_Label_encoder.transform(df["data"])
df["y"]=y

In [105]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(df,x="temperature", y="absorbance99",color="y", hover_name="donation_id")
fig.show()

In [106]:
func=["mean"]

In [107]:
df.columns

Index(['absorbance0', 'absorbance1', 'absorbance2', 'absorbance3',
       'absorbance4', 'absorbance5', 'absorbance6', 'absorbance7',
       'absorbance8', 'absorbance9',
       ...
       'id', 'std', 'temperature', 'humidity', 'hdl_cholesterol_human',
       'hemoglobin(hgb)_human', 'cholesterol_ldl_human', 'donation_id', 'data',
       'y'],
      dtype='object', length=180)

In [108]:
cols=list(df.columns[:170])
cols.append("y")
df_agg=pd.DataFrame()
for col in cols:
   
    result=df.groupby("donation_id").agg({col:func})
   
    df_agg[col]=result[col].iloc[:,0].values

In [109]:
df_agg.tail()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,y
477,0.497722,0.502236,0.506524,0.512087,0.518915,0.524930,0.530041,0.534502,0.542757,0.552737,0.565028,0.574349,0.579139,0.581125,0.581965,0.580853,0.579013,0.576255,0.573284,0.568689,0.565339,0.561658,0.557967,0.554181,0.549629,0.546424,0.543400,0.540834,0.538592,0.535892,0.534333,0.533390,0.532452,0.532499,0.532923,0.534765,0.536713,0.539096,0.541816,0.544660,...,1.379679,1.369544,1.360657,1.348326,1.339978,1.332219,1.323447,1.314997,1.304944,1.296910,1.288176,1.280572,1.272800,1.264280,1.255728,1.249295,1.243557,1.236604,1.229171,1.224408,1.218067,1.212621,1.209260,1.202567,1.200033,1.196816,1.194646,1.192494,1.193658,1.194338,1.191805,1.186524,1.176843,1.162926,1.141868,1.127793,1.116824,1.117265,1.120076,20
478,0.538159,0.543001,0.546593,0.554427,0.561197,0.568280,0.573260,0.576354,0.585874,0.595715,0.607556,0.617710,0.622267,0.624192,0.624809,0.624153,0.621749,0.619409,0.615999,0.612229,0.608225,0.605031,0.601259,0.597529,0.593548,0.590421,0.587813,0.585632,0.583786,0.581461,0.580251,0.579488,0.579324,0.579680,0.580863,0.582662,0.584779,0.588078,0.590822,0.594678,...,1.588874,1.572394,1.557872,1.540459,1.528486,1.515093,1.503353,1.491938,1.476718,1.465122,1.454285,1.442681,1.433186,1.420192,1.409716,1.401413,1.392733,1.385906,1.375892,1.367697,1.358829,1.353254,1.348037,1.339421,1.334817,1.329508,1.326188,1.323609,1.322396,1.323488,1.318589,1.311474,1.300258,1.281109,1.256339,1.238991,1.235110,1.233912,1.243903,21
479,0.520465,0.523494,0.527131,0.533695,0.539521,0.546053,0.552104,0.557280,0.569123,0.580932,0.594431,0.605667,0.611095,0.613536,0.614940,0.614643,0.612586,0.610194,0.606941,0.603056,0.598837,0.595483,0.591545,0.587862,0.583802,0.580405,0.578077,0.575785,0.573999,0.571891,0.570904,0.570315,0.570521,0.571219,0.572613,0.575043,0.577676,0.581193,0.584609,0.588756,...,1.528357,1.516232,1.503433,1.488898,1.477528,1.466720,1.457065,1.446306,1.434634,1.423179,1.412889,1.403214,1.394982,1.383024,1.375603,1.365959,1.358637,1.351012,1.343245,1.335265,1.329645,1.322139,1.317480,1.310839,1.307073,1.303006,1.300065,1.297397,1.296561,1.295112,1.293383,1.291404,1.284806,1.277044,1.266703,1.256625,1.261502,1.256473,1.263625,4
480,0.565816,0.570778,0.575131,0.580975,0.588246,0.595434,0.599804,0.603518,0.611050,0.620812,0.631647,0.639707,0.643909,0.645798,0.646228,0.644960,0.643130,0.640030,0.636866,0.632735,0.629088,0.625793,0.621891,0.618315,0.614184,0.611281,0.608477,0.606190,0.604054,0.602135,0.600501,0.599825,0.599316,0.599694,0.600263,0.601852,0.603785,0.606329,0.608977,0.612608,...,1.535708,1.522506,1.509586,1.494082,1.482416,1.470886,1.459718,1.449305,1.436079,1.425227,1.415581,1.404063,1.394745,1.382932,1.374558,1.365291,1.358071,1.350134,1.340359,1.332457,1.326487,1.319904,1.313570,1.306074,1.301604,1.296310,1.292912,1.289982,1.288386,1.286577,1.283421,1.273842,1.259816,1.240772,1.210779,1.192688,1.181876,1.175161,1.174429,6
481,0.528848,0.528815,0.535568,0.540085,0.548474,0.

In [110]:
X_columns=df_agg.columns[:-1]
y_columns=df_agg.columns[-1]

In [111]:
X=df_agg[X_columns]
y=df_agg[y_columns]


In [112]:
X

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169
0,0.505055,0.506570,0.511544,0.516300,0.524067,0.530191,0.536138,0.542317,0.551118,0.567411,0.582043,0.594632,0.601357,0.606002,0.610174,0.610535,0.607241,0.602593,0.598158,0.594082,0.589946,0.586214,0.582763,0.579071,0.576070,0.572933,0.569674,0.567799,0.566049,0.564866,0.564235,0.563938,0.564077,0.565843,0.567793,0.570145,0.573502,0.578465,0.583251,0.587553,...,1.735472,1.714144,1.697098,1.682700,1.665354,1.651634,1.635162,1.621304,1.608060,1.594216,1.580074,1.566603,1.551025,1.539729,1.531069,1.514383,1.502906,1.490929,1.480540,1.471343,1.463302,1.453560,1.445046,1.434967,1.428055,1.423711,1.417191,1.414312,1.410027,1.409971,1.410790,1.410953,1.410468,1.409675,1.401169,1.395687,1.390813,1.384304,1.395545,1.369307
1,0.523531,0.523485,0.526291,0.532875,0.538397,0.543180,0.547520,0.553433,0.561674,0.575610,0.588949,0.599821,0.606795,0.611225,0.614827,0.615984,0.613867,0.609749,0.605598,0.601913,0.598477,0.594828,0.591709,0.588105,0.585093,0.581967,0.579102,0.577092,0.574750,0.573190,0.572159,0.571213,0.571900,0.572428,0.573744,0.575815,0.578596,0.582865,0.586430,0.590458,...,1.564144,1.547562,1.533375,1.519141,1.506340,1.493741,1.479157,1.468128,1.456205,1.444356,1.433832,1.421925,1.409270,1.398967,1.391729,1.378697,1.369708,1.358800,1.350754,1.342818,1.337081,1.329671,1.321694,1.314480,1.309197,1.304323,1.300707,1.296805,1.294782,1.293546,1.295560,1.296753,1.298688,1.301346,1.299807,1.299004,1.298251,1.297575,1.302398,1.284716
2,0.516449,0.519226,0.523619,0.532366,0.540438,0.547985,0.552716,0.556823,0.561827,0.575226,0.588143,0.598764,0.604886,0.607795,0.610482,0.611734,0.609181,0.604104,0.599697,0.595304,0.590524,0.587276,0.583843,0.580398,0.577492,0.574359,0.572048,0.569875,0.568215,0.566841,0.566002,0.565557,0.566380,0.567466,0.568986,0.571264,0.574136,0.578393,0.582481,0.586298,...,1.630452,1.607749,1.589105,1.572394,1.556925,1.540884,1.523277,1.508105,1.493718,1.480541,1.466031,1.452204,1.436455,1.423938,1.413727,1.398970,1.387043,1.375768,1.365806,1.358372,1.348132,1.339412,1.331781,1.323498,1.315743,1.311629,1.306184,1.302404,1.297868,1.297409,1.295682,1.291848,1.283621,1.272086,1.247736,1.228340,1.211511,1.204599,1.212862,1.193458
3,0.512397,0.514371,0.518486,0.520868,0.526911,0.534963,0.542428,0.549013,0.556389,0.567360,0.580468,0.590795,0.598742,0.601499,0.602390,0.602226,0.601315,0.599644,0.595954,0.593102,0.589762,0.587078,0.583916,0.579818,0.576588,0.573678,0.571054,0.568825,0.567245,0.565668,0.565298,0.565279,0.565519,0.566874,0.568391,0.571296,0.574477,0.577985,0.581670,0.585977,...,1.496303,1.485007,1.476102,1.464696,1.454462,1.444651,1.433007,1.424728,1.416100,1.406215,1.397159,1.388156,1.377964,1.368401,1.362904,1.350944,1.344366,1.337207,1.329001,1.320706,1.315186,1.307761,1.303607,1.296942,1.291493,1.286922,1.282958,1.278915,1.278285,1.278112,1.274949,1.271895,1.268378,1.262765,1.254238,1.246901,1.248287,1.245556,1.266079,1.258333
4,0.496008,0.496698,0

In [113]:
import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences

In [114]:
X

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance130,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169
0,0.505055,0.506570,0.511544,0.516300,0.524067,0.530191,0.536138,0.542317,0.551118,0.567411,0.582043,0.594632,0.601357,0.606002,0.610174,0.610535,0.607241,0.602593,0.598158,0.594082,0.589946,0.586214,0.582763,0.579071,0.576070,0.572933,0.569674,0.567799,0.566049,0.564866,0.564235,0.563938,0.564077,0.565843,0.567793,0.570145,0.573502,0.578465,0.583251,0.587553,...,1.735472,1.714144,1.697098,1.682700,1.665354,1.651634,1.635162,1.621304,1.608060,1.594216,1.580074,1.566603,1.551025,1.539729,1.531069,1.514383,1.502906,1.490929,1.480540,1.471343,1.463302,1.453560,1.445046,1.434967,1.428055,1.423711,1.417191,1.414312,1.410027,1.409971,1.410790,1.410953,1.410468,1.409675,1.401169,1.395687,1.390813,1.384304,1.395545,1.369307
1,0.523531,0.523485,0.526291,0.532875,0.538397,0.543180,0.547520,0.553433,0.561674,0.575610,0.588949,0.599821,0.606795,0.611225,0.614827,0.615984,0.613867,0.609749,0.605598,0.601913,0.598477,0.594828,0.591709,0.588105,0.585093,0.581967,0.579102,0.577092,0.574750,0.573190,0.572159,0.571213,0.571900,0.572428,0.573744,0.575815,0.578596,0.582865,0.586430,0.590458,...,1.564144,1.547562,1.533375,1.519141,1.506340,1.493741,1.479157,1.468128,1.456205,1.444356,1.433832,1.421925,1.409270,1.398967,1.391729,1.378697,1.369708,1.358800,1.350754,1.342818,1.337081,1.329671,1.321694,1.314480,1.309197,1.304323,1.300707,1.296805,1.294782,1.293546,1.295560,1.296753,1.298688,1.301346,1.299807,1.299004,1.298251,1.297575,1.302398,1.284716
2,0.516449,0.519226,0.523619,0.532366,0.540438,0.547985,0.552716,0.556823,0.561827,0.575226,0.588143,0.598764,0.604886,0.607795,0.610482,0.611734,0.609181,0.604104,0.599697,0.595304,0.590524,0.587276,0.583843,0.580398,0.577492,0.574359,0.572048,0.569875,0.568215,0.566841,0.566002,0.565557,0.566380,0.567466,0.568986,0.571264,0.574136,0.578393,0.582481,0.586298,...,1.630452,1.607749,1.589105,1.572394,1.556925,1.540884,1.523277,1.508105,1.493718,1.480541,1.466031,1.452204,1.436455,1.423938,1.413727,1.398970,1.387043,1.375768,1.365806,1.358372,1.348132,1.339412,1.331781,1.323498,1.315743,1.311629,1.306184,1.302404,1.297868,1.297409,1.295682,1.291848,1.283621,1.272086,1.247736,1.228340,1.211511,1.204599,1.212862,1.193458
3,0.512397,0.514371,0.518486,0.520868,0.526911,0.534963,0.542428,0.549013,0.556389,0.567360,0.580468,0.590795,0.598742,0.601499,0.602390,0.602226,0.601315,0.599644,0.595954,0.593102,0.589762,0.587078,0.583916,0.579818,0.576588,0.573678,0.571054,0.568825,0.567245,0.565668,0.565298,0.565279,0.565519,0.566874,0.568391,0.571296,0.574477,0.577985,0.581670,0.585977,...,1.496303,1.485007,1.476102,1.464696,1.454462,1.444651,1.433007,1.424728,1.416100,1.406215,1.397159,1.388156,1.377964,1.368401,1.362904,1.350944,1.344366,1.337207,1.329001,1.320706,1.315186,1.307761,1.303607,1.296942,1.291493,1.286922,1.282958,1.278915,1.278285,1.278112,1.274949,1.271895,1.268378,1.262765,1.254238,1.246901,1.248287,1.245556,1.266079,1.258333
4,0.496008,0.496698,0

In [115]:
def blood_model():
    #clear_session()



    input_len=170

    print(input_len)
    output_size=24
    drop_frac0=0.0 
    drop_frac1=0.0



    input1=Input(shape=(input_len,),)

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="normal"
    d1=Dense(113,activation="selu",kernel_initializer=initializer,input_dim=input_len)(input1)
    d1=Dropout(drop_frac0)(d1)
    d1=Dense(16,activation="selu",kernel_initializer=initializer,input_dim=input_len, name="bottle")(d1)
    d1=Dropout(drop_frac0)(d1)
    d1=Dense(113,activation="selu",kernel_initializer=initializer,input_dim=input_len)(d1)
    d1=Dropout(drop_frac0)(d1)
    



    pred=Dense(input_len,activation="selu")(d1)

    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.Adam(learning_rate=0.001)


    lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="mse",
        optimizer=opt,
        metrics=["mse"])
    return(model)

In [116]:
print(max(y))

23


In [117]:
model_name="Newdata_col"
def scheduler(epoch, lr):
    return 0.000005

callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

callbacks = [callback_LR,
        
        #savemodela,
        ModelCheckpoint(filepath=model_name+"_{mse:.5f}_{val_mse:.5f}_.hdf5", monitor='val_mse',
                        verbose=1, save_best_only=True, mode='min')]

In [118]:
new_model=blood_model()

170


In [163]:
new_model.fit(X,
          X,
          epochs=1250, 
          batch_size=3,
          validation_split=0.1,
          verbose=1,
          callbacks=callbacks,
          shuffle=True
          
          )

Epoch 1/1250
134/145 [==========================>...] - ETA: 0s - loss: 3.9294e-05 - mse: 3.9294e-05
Epoch 00001: val_mse did not improve from 0.00005
145/145 [==============================] - 1s 4ms/step - loss: 3.8653e-05 - mse: 3.8653e-05 - val_loss: 5.1037e-05 - val_mse: 5.1037e-05 - lr: 5.0000e-06
Epoch 2/1250
125/145 [========================>.....] - ETA: 0s - loss: 3.9356e-05 - mse: 3.9356e-05
Epoch 00002: val_mse did not improve from 0.00005
145/145 [==============================] - 0s 2ms/step - loss: 3.8972e-05 - mse: 3.8972e-05 - val_loss: 4.7685e-05 - val_mse: 4.7685e-05 - lr: 5.0000e-06
Epoch 3/1250
136/145 [===========================>..] - ETA: 0s - loss: 3.8764e-05 - mse: 3.8764e-05
Epoch 00003: val_mse did not improve from 0.00005
145/145 [==============================] - 0s 2ms/step - loss: 3.8782e-05 - mse: 3.8782e-05 - val_loss: 4.9700e-05 - val_mse: 4.9700e-05 - lr: 5.0000e-06
Epoch 4/1250
132/145 [==========================>...] - ETA: 0s - loss: 3.7916e-05 - 

In [ ]:
X

In [167]:
new_model.load_weights( "Newdata_col_0.00002_0.00003_.hdf5")

In [168]:
new_model_bottle=Model(new_model.inputs,new_model.get_layer("bottle").output)
new_pred_bottle=new_model_bottle.predict(X)

In [169]:
new_pred_bottle=new_model_bottle.predict(X)

In [315]:
pca_df=pd.DataFrame(new_pred_bottle)
pca_df.columns=["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15"]

In [316]:
pca_df["y"]=df_agg[y_columns]

In [332]:
pca_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,y
0,-0.455455,-0.210233,4.744530,-0.163626,4.947189,-0.305293,1.694035,-0.139085,-0.405653,2.794690,3.937781,-0.753464,1.214020,4.544711,0.394087,4.037780,14
1,-0.487165,-0.134975,4.500106,-0.198809,4.477697,-0.131440,1.613412,-0.107356,-0.347692,2.657871,3.544773,-0.688633,1.265965,4.172063,0.649278,3.789781,12
2,-0.581075,-0.397844,4.716376,-0.013440,4.517286,-0.133377,1.670402,-0.101666,-0.316298,2.734718,3.493757,-0.700843,1.259092,4.410285,0.424075,3.868018,12
3,-0.427012,-0.037865,4.373947,-0.202987,4.316164,-0.199464,1.531728,-0.082108,-0.290962,2.587633,3.431535,-0.654287,1.299849,4.001109,0.803744,3.675647,12
4,-0.485447,-0.119334,4.234157,-0.105513,4.203715,-0.118692,1.498959,-0.099179,-0.329787,2.463407,3.329521,-0.635593,1.166434,3.930183,0.577152,3.545022,14


In [333]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(pca_df,x="3", y="4",color="y")
fig.show()

PCA dimenzió csökkentés

In [349]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(pca_df.iloc[:,:-1])


PCA(n_components=3)

In [350]:
pca_df.iloc[:,:-1]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.455455,-0.210233,4.744530,-0.163626,4.947189,-0.305293,1.694035,-0.139085,-0.405653,2.794690,3.937781,-0.753464,1.214020,4.544711,0.394087,4.037780
1,-0.487165,-0.134975,4.500106,-0.198809,4.477697,-0.131440,1.613412,-0.107356,-0.347692,2.657871,3.544773,-0.688633,1.265965,4.172063,0.649278,3.789781
2,-0.581075,-0.397844,4.716376,-0.013440,4.517286,-0.133377,1.670402,-0.101666,-0.316298,2.734718,3.493757,-0.700843,1.259092,4.410285,0.424075,3.868018
3,-0.427012,-0.037865,4.373947,-0.202987,4.316164,-0.199464,1.531728,-0.082108,-0.290962,2.587633,3.431535,-0.654287,1.299849,4.001109,0.803744,3.675647
4,-0.485447,-0.119334,4.234157,-0.105513,4.203715,-0.118692,1.498959,-0.099179,-0.329787,2.463407,3.329521,-0.635593,1.166434,3.930183,0.577152,3.545022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,-0.432547,-0.042612,4.151781,-0.113563,3.968148,-0.173789,1.403959,-0.074802,-0.228070,2.423725,3.142492,-0.593862,1.267659,3.745988,0.834010,3.430320
478,-0.545170,-0.269155,4.665953,-0.103953,4.508532,-0.180602,1.628420,-0.085084,-0.291235,2.718650,3.552463,-0.698947,1.317805,4.318055,0.604945,3.860862
479,-0.459478,-0.136953,4.505611,-0.172796,4.410958,-0.180601,1.582785,-0.114629,-0.298181,2.668736,3.473046,-0.682502,1.312668,4.148006,0.727984,3.771098
480,-0.578147,-0.267140,4.660335,-0.110601,4.314273,-0.134886,1.603399,-0.049904,-0.222459,2.707603,3.400439,-0.674179,1.411373,4.214275,0.782107,3.800989


In [351]:
pca_out=pca.transform(pca_df.iloc[:,:-1])
pca_out_df=pd.DataFrame(pca_out)
pca_out_df["y"]=pca_df["y"]

In [352]:
pca_out_df.head()

,0,1,2,y
0,-0.937859,-0.175943,-0.025158,14
1,-0.097903,0.012702,-0.032255,12
2,-0.369259,-0.112322,0.410119,12
3,0.257648,0.112359,-0.128117,12
4,0.482845,-0.211820,-0.022614,14


In [353]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(pca_out_df,x=0, y=1,color="y")
fig.show()

#Random Forest

In [450]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score


clf = RandomForestClassifier(max_depth=8, random_state=0)
clf.fit(X, y)

randomforest_predict=clf.predict(X)

In [452]:
print(f"Accuracy: {accuracy_score(y,randomforest_predict)}")

Accuracy: 0.8713692946058091


In [453]:
y

0      14
1      12
2      12
3      12
4      14
       ..
477    20
478    21
479     4
480     6
481     4
Name: y, Length: 482, dtype: int64

In [454]:
new_pred_bottle

array([[-0.4554547 , -0.21023284,  4.7445297 , ...,  4.544711  ,
         0.39408657,  4.03778   ],
       [-0.48716477, -0.13497454,  4.500106  , ...,  4.172063  ,
         0.64927834,  3.7897806 ],
       [-0.58107466, -0.39784446,  4.716376  , ...,  4.410285  ,
         0.42407537,  3.868018  ],
       ...,
       [-0.45947847, -0.13695267,  4.505611  , ...,  4.1480064 ,
         0.727984  ,  3.771098  ],
       [-0.5781468 , -0.26713985,  4.660335  , ...,  4.2142754 ,
         0.7821071 ,  3.8009887 ],
       [-0.5575501 , -0.28805977,  4.5638456 , ...,  4.157392  ,
         0.6810898 ,  3.7408068 ]], dtype=float32)

In [455]:
def cosine_similarity(x, y):
    import numpy as np
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [456]:
# def a search matrix
def create_search_list(pred,df,str_index):
    out=[]
    for i,vec in enumerate(pred):
        out.append([vec,df[str_index].values[i]])
    return(out)
    
    

In [457]:
__DELTA__=0.00005
def generate_similarity(it1,sim_table):
    out=[]
    for it2 in sim_table:
        
        diff=cosine_similarity(it1,it2[0])
        if 1-diff<__DELTA__:
            
            out.append(it2[1])
    return(out)

In [458]:
def find_max(dict):
    max=0
    sum=0
    outkey=""
    for key in dict.keys():
        akt=dict[key]
        sum+=akt
        if max<akt:
            max=akt
            outkey=key
    return outkey,max

In [459]:
def statistic(a):
    c=list(set(a))
    #print(c)
    out={}
    for i in c:
        out[i]=a.count(i)
    return(out)

In [460]:
new_bottle_list=create_search_list(new_pred_bottle,df,"y")

In [461]:
new_bottle_list

[[array([-0.4554547 , -0.21023284,  4.7445297 , -0.16362615,  4.947189  ,
         -0.30529305,  1.6940346 , -0.13908549, -0.40565306,  2.79469   ,
          3.9377806 , -0.75346386,  1.21402   ,  4.544711  ,  0.39408657,
          4.03778   ], dtype=float32), 12],
 [array([-0.48716477, -0.13497454,  4.500106  , -0.19880906,  4.4776974 ,
         -0.13144025,  1.6134118 , -0.10735638, -0.34769216,  2.6578715 ,
          3.5447726 , -0.68863255,  1.2659647 ,  4.172063  ,  0.64927834,
          3.7897806 ], dtype=float32), 12],
 [array([-0.58107466, -0.39784446,  4.716376  , -0.01344048,  4.5172863 ,
         -0.13337731,  1.6704023 , -0.10166623, -0.31629798,  2.7347178 ,
          3.4937572 , -0.70084304,  1.259092  ,  4.410285  ,  0.42407537,
          3.868018  ], dtype=float32), 12],
 [array([-0.42701238, -0.03786451,  4.373947  , -0.20298707,  4.3161645 ,
         -0.19946422,  1.5317276 , -0.08210754, -0.29096153,  2.5876327 ,
          3.4315352 , -0.6542869 ,  1.2998489 ,  4.001

In [462]:
# submission generation
!rm "./Updated_Test.csv"

In [463]:
!wget https://github.com/sipocz/Blood/raw/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv

--2021-12-11 12:01:53--  https://github.com/sipocz/Blood/raw/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv [following]
--2021-12-11 12:01:54--  https://raw.githubusercontent.com/sipocz/Blood/ae2dd49b40364eeb9e1821b70aa87ff332aca170/orig/Updated_Test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10371075 (9.9M) [text/plain]
Saving to: ‘Updated_Test.csv’

Updated_Test.csv    100%[===================>]   9.89M  --.-KB/s    in 0.08s   

2021-12-11 12:01:5

In [464]:
test_df=pd.read_csv("./Updated_Test.csv")
test_df.head()

test_df['hdl_cholesterol_human']=0
test_df["hemoglobin(hgb)_human"]=0
test_df["cholesterol_ldl_human"]=0

In [465]:
test_df.head()

,Reading_ID,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,...,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,id,std,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
0,ID_KWUAQLRM,ID_2982,0.517951,0.520508,0.526852,0.531611,0.536816,0.543828,0.547761,0.554379,0.565622,0.575762,0.590253,0.600692,0.605104,0.608611,0.608169,0.606985,0.606590,0.603814,0.600409,0.597802,0.593389,0.589956,0.585429,0.583659,0.578307,0.574303,0.571248,0.570985,0.568520,0.566437,0.564261,0.563907,0.563427,0.564336,0.565472,0.567827,0.570957,0.572206,...,1.385020,1.373256,1.362781,1.345200,1.341810,1.333717,1.328438,1.313325,1.305895,1.293996,1.292892,1.285608,1.275789,1.267682,1.263906,1.259461,1.249615,1.244618,1.242537,1.238426,1.233983,1.235212,1.234433,1.234971,1.223950,1.235415,1.234605,1.226695,1.210430,1.214422,1.200332,1.198297,1.248182,0,0.0027,37.09,27.22,0,0,0
1,ID_FN8SIVJJ,ID_2982,0.517839,0.522367,0.525186,0.534661,0.541900,0.546180,0.551687,0.556753,0.566446,0.578208,0.591039,0.601620,0.605895,0.609568,0.610188,0.608635,0.607716,0.606237,0.601746,0.597047,0.594703,0.591059,0.587887,0.582815,0.578746,0.576693,0.573624,0.571268,0.569092,0.567672,0.565814,0.564862,0.564965,0.565536,0.565475,0.567822,0.571052,0.572733,...,1.387243,1.377109,1.362596,1.350712,1.343471,1.334255,1.320115,1.316767,1.305150,1.289700,1.286014,1.284353,1.274027,1.274710,1.263676,1.255490,1.256831,1.246188,1.236320,1.242136,1.234657,1.233898,1.234317,1.232710,1.238981,1.237033,1.237152,1.219627,1.216220,1.191416,1.202515,1.211942,1.242939,1,0.0027,37.20,27.17,0,0,0
2,ID_CI4TQCHI,ID_2982,0.517702,0.522018,0.527237,0.534374,0.541155,0.547152,0.549837,0.557513,0.566793,0.580574,0.592258,0.601226,0.608085,0.611143,0.611085,0.608700,0.608647,0.605048,0.601668,0.598030,0.595072,0.591417,0.587822,0.584160,0.579395,0.576524,0.574158,0.572215,0.569473,0.566740,0.565881,0.565431,0.565380,0.565508,0.566487,0.568591,0.570932,0.573199,...,1.381880,1.377176,1.360407,1.350644,1.368027,1.338249,1.322779,1.313343,1.300640,1.295481,1.286947,1.283352,1.276702,1.270998,1.262507,1.259525,1.253063,1.247183,1.245595,1.248910,1.239302,1.236558,1.237110,1.239622,1.228925,1.240135,1.238346,1.237193,1.208477,1.208721,1.187810,1.209599,1.175631,2,0.0027,37.30,27.07,0,0,0
3,ID_URJMKNU7,ID_2982,0.525008,0.527439,0.536871,0.539636,0.546555,0.553183,0.558826,0.563549,0.575675,0.587214,0.597155,0.606784,0.612443,0.615604,0.616249,0.615097,0.614963,0.611250,0.608559,0.603462,0.599570,0.597532,0.593608,0.588978,0.584560,0.581973,0.578872,0.576858,0.573706,0.571340,0.570279,0.570198,0.568850,0.569583,0.570136,0.571026,0.574510,0.575988,...,1.382891,1.377412,1.357813,1.352627,1.344537,1.334463,1.323303,1.313057,1.304806,1.299465,1.296792,1.286711,1.273948,1.272122,1.260676,1.255276,1.257032,1.244043,1.246127,1.238651,1.236244,1.243770,1.237752,1.229684,1.228355,1.234852,1.220159,1.218532,1.185708,1.196750,1.222484,1.177406,1.198399,3,0.0027,37.38,26.98,0,0,0
4,ID_HBJH67IS,ID_2982,0.520532,0.522683,0.526842,0.534634,0.539676,0.547488,0.552688,0.558355,0.568959,0.578905,0.591207,0.602236,0.607339,0.611220,0.610289,0.60955

In [466]:
test_id_list_tmp=test_df.donation_id.values
test_id_list=[]
for id in test_id_list_tmp:
    if id not in test_id_list:
        test_id_list.append(id)


In [467]:
cols=list(test_df.columns[2:172])

df_test_agg=pd.DataFrame()
for col in cols:
   
    result=test_df.groupby("donation_id").agg({col:func})
   
    df_test_agg[col]=result[col].iloc[:,0].values
df_test_agg["donation_id"]=result.index

In [468]:
#result.index

In [469]:
#df_test_agg.head()

In [470]:
test_X=df_test_agg.iloc[:,0:170]

In [471]:
#df_test_agg.describe()

In [472]:
# calculate bottle vektor for test data
bottle_pred_test=new_model_bottle.predict(test_X)


In [473]:
#new_bottle_list

In [474]:
__DELTA__=0.00014
test_result=[]
db=0
for test_i in bottle_pred_test:
    o1=generate_similarity( test_i, new_bottle_list)
    print(statistic(o1), end="-->")
    result,_=find_max(statistic(o1))
    if result=="":
        result=12
        print("NINCS !!!")
    db+=1
    if (db % 50)==0:
        print("...")
    print(result,end=",")
    test_result.append(result)

{12: 3, 22: 1, 23: 3}-->12,{12: 9, 23: 4, 14: 4, 22: 8}-->12,{12: 11, 23: 7, 14: 5, 22: 5}-->12,{12: 4, 14: 1, 21: 1, 22: 5, 23: 4}-->22,{12: 1}-->12,{12: 4, 14: 2, 21: 1, 22: 5, 23: 4}-->22,{12: 8, 23: 8, 14: 5, 22: 15}-->22,{12: 12, 23: 3, 14: 3, 22: 3}-->12,{12: 9, 23: 4, 14: 3, 22: 7}-->12,{12: 13, 23: 5, 14: 2, 22: 10}-->12,{12: 3, 23: 1, 14: 2, 22: 3}-->12,{23: 3, 14: 1, 22: 2}-->23,{12: 12, 23: 11, 14: 2, 22: 7}-->12,{12: 2, 23: 2, 14: 1, 22: 5}-->22,{12: 2, 22: 2, 23: 6}-->23,{12: 4, 22: 1, 23: 1}-->12,{}-->NINCS !!!
12,{12: 3, 14: 3, 22: 3}-->12,{12: 13, 23: 7, 14: 3, 22: 10}-->12,{12: 17, 23: 17, 14: 8, 22: 21}-->22,{12: 14, 23: 2, 14: 3, 22: 7}-->12,{14: 1, 23: 2}-->23,{12: 5, 23: 4, 14: 5, 22: 4}-->12,{12: 4, 23: 3, 14: 2, 22: 7}-->22,{12: 17, 23: 11, 14: 3, 22: 8}-->12,{12: 20, 23: 9, 14: 8, 22: 19}-->12,{23: 4, 12: 4, 14: 1, 22: 4}-->23,{12: 6, 23: 5, 14: 1, 22: 7}-->22,{12: 15, 23: 15, 14: 4, 22: 15}-->12,{}-->NINCS !!!
12,{12: 6, 22: 4, 23: 3}-->12,{12: 1, 22: 3, 23: 1}

In [475]:
test_result_randomforest=clf.predict(test_X)

In [476]:
__OUT_GENERATOR__="Random Forest"

In [477]:
if __OUT_GENERATOR__=="Neural":
    pred=_3_Label_encoder.inverse_transform(test_result)
elif __OUT_GENERATOR__=="Random Forest":
    pred=_3_Label_encoder.inverse_transform(test_result_randomforest)
    

In [478]:
pred

array(['ok+ok+high', 'high+ok+ok', 'low+ok+ok', 'ok+ok+ok', 'ok+ok+high',
       'ok+ok+high', 'ok+ok+high', 'ok+ok+high', 'ok+ok+ok', 'ok+ok+high',
       'high+ok+high', 'ok+ok+ok', 'ok+ok+high', 'ok+ok+ok',
       'high+ok+high', 'ok+ok+ok', 'ok+ok+high', 'ok+ok+ok', 'ok+ok+ok',
       'ok+ok+high', 'low+ok+ok', 'low+ok+ok', 'ok+ok+ok', 'low+ok+ok',
       'ok+ok+ok', 'ok+ok+high', 'ok+ok+ok', 'ok+ok+ok', 'ok+ok+ok',
       'ok+ok+high', 'low+ok+ok', 'ok+ok+ok', 'ok+ok+high', 'ok+ok+ok',
       'ok+ok+high', 'ok+ok+high', 'ok+ok+ok', 'ok+ok+ok', 'ok+ok+ok',
       'ok+ok+high', 'ok+ok+high', 'low+ok+ok', 'low+ok+ok', 'ok+ok+ok',
       'ok+ok+high', 'ok+ok+ok', 'ok+ok+ok', 'ok+ok+ok', 'ok+ok+ok',
       'ok+ok+ok', 'ok+ok+ok', 'high+ok+ok', 'high+ok+ok', 'ok+ok+high',
       'ok+ok+ok', 'ok+ok+ok', 'ok+ok+ok', 'ok+ok+ok', 'ok+ok+high',
       'ok+ok+ok', 'ok+ok+high'], dtype=object)

In [479]:
def de_conv(pred):
    out0=[]
    out1=[]
    out2=[]

    for p in pred:
        if p=="NaN":
            p="ok+ok+ok"
        print(p)
        
        out0.append(p.split("+")[0])
        out1.append(p.split("+")[1])
        out2.append(p.split("+")[2])
        
    return(out0,out1,out2)


In [480]:
ret0,ret1,ret2=de_conv(pred)

ok+ok+high
high+ok+ok
low+ok+ok
ok+ok+ok
ok+ok+high
ok+ok+high
ok+ok+high
ok+ok+high
ok+ok+ok
ok+ok+high
high+ok+high
ok+ok+ok
ok+ok+high
ok+ok+ok
high+ok+high
ok+ok+ok
ok+ok+high
ok+ok+ok
ok+ok+ok
ok+ok+high
low+ok+ok
low+ok+ok
ok+ok+ok
low+ok+ok
ok+ok+ok
ok+ok+high
ok+ok+ok
ok+ok+ok
ok+ok+ok
ok+ok+high
low+ok+ok
ok+ok+ok
ok+ok+high
ok+ok+ok
ok+ok+high
ok+ok+high
ok+ok+ok
ok+ok+ok
ok+ok+ok
ok+ok+high
ok+ok+high
low+ok+ok
low+ok+ok
ok+ok+ok
ok+ok+high
ok+ok+ok
ok+ok+ok
ok+ok+ok
ok+ok+ok
ok+ok+ok
ok+ok+ok
high+ok+ok
high+ok+ok
ok+ok+high
ok+ok+ok
ok+ok+ok
ok+ok+ok
ok+ok+ok
ok+ok+high
ok+ok+ok
ok+ok+high


In [481]:
df_test_agg.head()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance131,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id
0,0.499920,0.504608,0.507929,0.519943,0.529032,0.536118,0.539690,0.541710,0.548590,0.560742,0.572227,0.582468,0.588715,0.591515,0.594874,0.596132,0.593921,0.589298,0.584735,0.580726,0.576208,0.573006,0.569422,0.566248,0.563401,0.560530,0.557530,0.556048,0.554297,0.553283,0.553039,0.552928,0.553576,0.555021,0.556709,0.559042,0.562160,0.567039,0.570653,0.574812,...,1.408521,1.397224,1.387323,1.376899,1.367692,1.356969,1.349031,1.339057,1.330981,1.321598,1.312683,1.302362,1.293830,1.287395,1.278004,1.269806,1.261939,1.255188,1.248585,1.242906,1.237545,1.231696,1.225090,1.220767,1.214857,1.210767,1.208588,1.204434,1.200936,1.195675,1.185186,1.165124,1.140465,1.098607,1.065527,1.044592,1.035440,1.041868,1.027759,ID_1961
1,0.497758,0.498091,0.503083,0.507249,0.513739,0.521944,0.528798,0.533743,0.541470,0.551838,0.563810,0.576655,0.582322,0.585335,0.587424,0.588745,0.588792,0.586696,0.584191,0.581720,0.579240,0.576917,0.573680,0.570807,0.567496,0.564498,0.562600,0.560363,0.558279,0.557036,0.555625,0.554756,0.554074,0.554820,0.555545,0.557092,0.559098,0.562553,0.565149,0.568692,...,1.426360,1.415701,1.404441,1.393965,1.381753,1.371568,1.363942,1.352835,1.344149,1.335032,1.323417,1.314956,1.306595,1.300673,1.289002,1.282047,1.274748,1.266909,1.260009,1.254758,1.247724,1.241378,1.236482,1.229732,1.225451,1.223462,1.218626,1.218425,1.217652,1.216998,1.218545,1.221584,1.221688,1.221872,1.219100,1.221694,1.221569,1.233970,1.227104,ID_2067
2,0.551436,0.553079,0.556722,0.560722,0.565583,0.570889,0.578668,0.585142,0.594233,0.610121,0.624801,0.637173,0.645108,0.649101,0.650867,0.651125,0.649180,0.647096,0.644185,0.640608,0.637902,0.635386,0.631675,0.625593,0.621151,0.616636,0.613757,0.610795,0.608635,0.607679,0.606690,0.606639,0.606520,0.608730,0.611250,0.614201,0.617529,0.621660,0.627568,0.632319,...,1.657361,1.644638,1.632069,1.616462,1.604119,1.593780,1.583286,1.571732,1.558965,1.547836,1.536559,1.527750,1.517924,1.511536,1.496439,1.490027,1.480749,1.474753,1.464195,1.456410,1.449595,1.442547,1.436186,1.430327,1.426222,1.421095,1.415937,1.416552,1.414947,1.412894,1.416760,1.419018,1.422387,1.421566,1.422355,1.423502,1.433594,1.451166,1.433315,ID_2155
3,0.525926,0.528948,0.532222,0.540022,0.547279,0.554348,0.558680,0.563768,0.570031,0.583540,0.596124,0.607093,0.613002,0.616624,0.619543,0.620841,0.618805,0.614166,0.610156,0.605965,0.601668,0.598687,0.595476,0.592006,0.589363,0.586510,0.583951,0.582185,0.580720,0.579559,0.578678,0.578325,0.579207,0.580438,0.581965,0.584198,0.587096,0.591386,0.595273,0.599130,...,1.541105,1.526136,1.511591,1.497848,1.483961,1.468861,1.456634,1.442976,1.431223,1.419422,1.407361,1.394721,1.383817,1.375959,1.362057,1.353425,1.343169,1.336277,1.328247,1.320609,1.312814,1.306433,1.298607,1.291746,1.287399,1.283340,1.279884,1.276867,1.275387,1.273222,1.271094,1.264453,1.251708,1.234305,1.212873,1.203968,1.196735,1.201349,1.192471,ID_2211
4,0.559814,0.559612,0.56156

In [482]:
outdf=pd.DataFrame()
outdf["Reading_ID"]=df_test_agg["donation_id"]
outdf["donation_id"]=df_test_agg.donation_id
outdf["hdl_cholesterol_human"]=ret0
outdf["hemoglobin(hgb)_human"]=ret1
outdf["cholesterol_ldl_human"]=ret2

outdf.set_index("Reading_ID",inplace=True)

In [483]:
outdf

,donation_id,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
Reading_ID,,,,
ID_1961,ID_1961,ok,ok,high
ID_2067,ID_2067,high,ok,ok
ID_2155,ID_2155,low,ok,ok
ID_2211,ID_2211,ok,ok,ok
ID_2327,ID_2327,ok,ok,high
...,...,...,...,...
ID_6373,ID_6373,ok,ok,ok
ID_6381,ID_6381,ok,ok,ok
ID_6391,ID_6391,ok,ok,high


In [484]:
gen_outdf=pd.DataFrame()
c1=[]
c2=[]
for i in outdf.index:
    rec1=outdf.loc[i,"hdl_cholesterol_human"]
    rec2=outdf.loc[i,"hemoglobin(hgb)_human"]
    rec3=outdf.loc[i,"cholesterol_ldl_human"]
    c1.append(outdf.loc[i,"donation_id"]+"_"+"hdl_cholesterol_human")
    c2.append(rec1)
    c1.append(outdf.loc[i,"donation_id"]+"_"+"hemoglobin(hgb)_human")
    c2.append(rec2)
    c1.append(outdf.loc[i,"donation_id"]+"_"+"cholesterol_ldl_human")
    c2.append(rec3)
    
    

In [485]:
print (c1)

['ID_1961_hdl_cholesterol_human', 'ID_1961_hemoglobin(hgb)_human', 'ID_1961_cholesterol_ldl_human', 'ID_2067_hdl_cholesterol_human', 'ID_2067_hemoglobin(hgb)_human', 'ID_2067_cholesterol_ldl_human', 'ID_2155_hdl_cholesterol_human', 'ID_2155_hemoglobin(hgb)_human', 'ID_2155_cholesterol_ldl_human', 'ID_2211_hdl_cholesterol_human', 'ID_2211_hemoglobin(hgb)_human', 'ID_2211_cholesterol_ldl_human', 'ID_2327_hdl_cholesterol_human', 'ID_2327_hemoglobin(hgb)_human', 'ID_2327_cholesterol_ldl_human', 'ID_2436_hdl_cholesterol_human', 'ID_2436_hemoglobin(hgb)_human', 'ID_2436_cholesterol_ldl_human', 'ID_2441_hdl_cholesterol_human', 'ID_2441_hemoglobin(hgb)_human', 'ID_2441_cholesterol_ldl_human', 'ID_2493_hdl_cholesterol_human', 'ID_2493_hemoglobin(hgb)_human', 'ID_2493_cholesterol_ldl_human', 'ID_2495_hdl_cholesterol_human', 'ID_2495_hemoglobin(hgb)_human', 'ID_2495_cholesterol_ldl_human', 'ID_2517_hdl_cholesterol_human', 'ID_2517_hemoglobin(hgb)_human', 'ID_2517_cholesterol_ldl_human', 'ID_2524_

In [486]:
gen_outdf["Donation_ID"]=c1
gen_outdf["target"]=c2


In [487]:
gen_outdf.head()

,Donation_ID,target
0,ID_1961_hdl_cholesterol_human,ok
1,ID_1961_hemoglobin(hgb)_human,ok
2,ID_1961_cholesterol_ldl_human,high
3,ID_2067_hdl_cholesterol_human,high
4,ID_2067_hemoglobin(hgb)_human,ok


In [488]:
def create_submission_name(project):
    from datetime  import datetime
    strdate=datetime.now().strftime("%Y%m%d_%H%M%S")
    out=project+"_"+"submission"+"_"+strdate+".csv"
    return(out)

In [489]:
gen_outdf.set_index("Donation_ID", inplace=True)

In [490]:
submission_fname=create_submission_name("BloodAI")

In [491]:
submission_fname

'BloodAI_submission_20211211_120156.csv'

In [492]:
gen_outdf.to_csv(submission_fname,index=True)

In [493]:
!head $submission_fname

Donation_ID,target
ID_1961_hdl_cholesterol_human,ok
ID_1961_hemoglobin(hgb)_human,ok
ID_1961_cholesterol_ldl_human,high
ID_2067_hdl_cholesterol_human,high
ID_2067_hemoglobin(hgb)_human,ok
ID_2067_cholesterol_ldl_human,ok
ID_2155_hdl_cholesterol_human,low
ID_2155_hemoglobin(hgb)_human,ok
ID_2155_cholesterol_ldl_human,ok
